For first submission,
following were the features
1. Tfidf
2. unigram PMI lexicon features

## Importing Libraries

In [1]:
import numpy as np
import pandas as pd
import gensim
import logging
import re
import nltk.data
from nltk.corpus import stopwords
import pysentiment as ps
#from practnlptools.tools import Annotator
from nltk.stem.porter import *
stemmer = PorterStemmer() 
from __future__ import division
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
import codecs

C:\Users\Zarmeen\Anaconda2\envs\gl-env\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
import sys
reload(sys)
sys.setdefaultencoding('utf-8')

In [3]:
stops = set(stopwords.words("english"))


## Reading Headlines Dataset

In [4]:
training_data = pd.read_json(codecs.open('Headline_Trainingdata.json', 'r', 'utf-8'),orient='records')
test_data = pd.read_json(codecs.open('Headlines_Testdata.json', 'r', 'utf-8'),orient='records')

In [6]:
test_data.shape

(491, 3)

In [7]:
training_data.shape

(1142, 4)

In [8]:
training_data.head()

,company,id,sentiment,title
0,Morrisons,2,0.430,Morrisons book second consecutive quarter of s...
1,IMI,3,-0.344,IMI posts drop in first-quarter organic revenu...
2,Glencore,4,0.340,Glencore to refinance its short-term debt earl...
3,Ryanair,5,0.259,EasyJet attracts more passengers in June but s...
4,Barclays,6,-0.231,Barclays 'bad bank' chief to step down


In [9]:
test_data.head()

,company,id,title
0,Ashtead,1144,"Ashtead to buy back shares, full-year profit b..."
1,Shell,1145,EU regulators clear Shell's takeover of BG Group
2,Prudential,1146,UK's FTSE has worst day so far in 2015 as BG a...
3,GlaxoSmithKline,1147,GlaxoSmithKline acquires HIV assets
4,Barclays,1148,Barclays faces another heavy forex fine


## Reading Lexicons

In [11]:
words_lex = []
score_lex = []
def lex_SCL():
    fname = 'SCL-NMA.txt'
    f = open(fname)
    for l in f.readlines():
       
        word,score = l.split('\t')
        words_lex.append(word)
        score = score.replace('\n','')
        score_lex.append(float(score))
    return words_lex,score_lex


In [12]:
words_lex_pmi = []
score_lex_pmi = []
def lex_PMI():
    fname = 'unigrams-pmilexicon.txt'
    f = open(fname)
    for l in f.readlines():
       
        word,score,pos,neg = l.split('\t')
        #print word
        #print word.startswith('@')
        if(word.startswith('@') == False and (word[0:3] != 'http')):
            words_lex_pmi.append(word)
            #score = score.replace('\n','')
            score_lex_pmi.append(float(score))
      
    return words_lex_pmi,score_lex_pmi


In [13]:
words_lex,score_lex = lex_SCL()
words_lex_pmi,score_lex_pmi = lex_PMI()

In [14]:
len(words_lex_pmi)

44668

## Preprocessing Text

In [20]:
def cleanText(text,companyName):
    text = re.sub("[^a-zA-Z]"," ", text)
    company = companyName.split(' ')
    words = text.split()
    words = [w for w in words if not w in stops and len(w) > 1 and w not in company]
    return ( " ".join(words) )

In [23]:
# function for computing lexicon score
def lex_score(text):
    tokens = text.split(' ')
    score = 0
    for word in tokens:
        for index in range(0,len(words_lex)):
            if(word.isupper() == False):
                if stemmer.stem(str(word.lower())) == stemmer.stem(words_lex[index]):
                        score = score+score_lex[index]
                        #print score
                        #print word
                        break

    return score

In [24]:
def lex_score_pmi(text):
    text = re.sub("[^a-zA-Z]"," ", text)
    text = re.sub('-',' ',text)
    text = re.sub("'",'',text)
    words = nltk.word_tokenize(text)
    tags = nltk.pos_tag(words)
    score = 0
    print tags
    for word in tags:
        if word[1] in ['VBD','JJR','JJ','VB','VBS','VBZ','VBN','VBG']:
            print word[0]
            for index in range(0,len(words_lex_pmi)):
                if(word[0].isupper() == False):
                    if word[0].lower() == words_lex_pmi[index].decode('utf-8'):
                            score = score+score_lex_pmi[index]
                            print words_lex_pmi[index]
                            print score_lex_pmi[index]

                            break

    return score

In [26]:
# Computing Harvard Lexicon scores
hiv4 = ps.HIV4()
def lex_score2(text):
    tokens = hiv4.tokenize(text)  # text can be tokenized by other ways
                                  # however, dict in HIV4 is preprocessed
                                  # by the default tokenizer in the library
    score = hiv4.get_score(tokens)

    return score['Polarity']

In [27]:
training_data['lex_score3'] = training_data['title'].apply(lex_score_pmi)

In [28]:
training_data['lex_score2'] = training_data['title'].apply(lex_score2)

In [29]:
test_data['lex_score3'] = test_data['title'].apply(lex_score_pmi)
test_data['lex_score2'] = test_data['title'].apply(lex_score2)

In [30]:
training_data[['sentiment','lex_score2','lex_score3']].head()

,sentiment,lex_score2,lex_score3
0,0.430,0.000000,0.077
1,-0.344,0.333333,1.304
2,0.340,0.999999,0.334
3,0.259,0.000000,1.019
4,-0.231,-0.999999,-0.909


## Splitting Training Dataset into Train/Test for Internal Evaluation

In [107]:
X_train, X_test, y_train, y_test = train_test_split(training_data[['id','title','lex_score3','lex_score2']],training_data[['sentiment']], test_size=0.20, random_state=42)


In [109]:
from sklearn.metrics import make_scorer,r2_score
from sklearn.metrics.pairwise import cosine_similarity

In [110]:
def customerscorer(y,y_pred):
    return cosine_similarity([y],[y_pred])

In [135]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5,
                                 stop_words='english')
X_training = vectorizer.fit_transform(X_train.title)
X_testing = vectorizer.transform(X_test.title)

In [88]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1, 3), max_df=0.4,
                                 stop_words='english')
X_training = vectorizer.fit_transform(X_train.title)
X_testing = vectorizer.transform(X_test.title)

In [136]:
x_params = { 'max_depth':range(3,10,2),
         'n_estimators': [80,100,150,200],
        'subsample':[i/100.0 for i in range(40,90,5)],
        'min_child_weight' :range(1,6,2),
        'colsample_bytree' :[i/100.0 for i in range(40,90,5)],
        'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100],
        'learning_rate':[0.1,0.5],
        'objective':['reg:linear'],
        'scale_pos_weight' : [1,5,10,50,100]
       }
xgb_model = xgb.XGBRegressor()

In [137]:
X_training.shape

(913, 2247)

In [138]:
X_testing.shape

(229, 2247)

In [148]:
train_feats =np.column_stack((X_training.toarray(),X_train['lex_score3']))
test_feats =np.column_stack((X_testing.toarray(),X_test['lex_score3']))


In [149]:
clf = RandomizedSearchCV(xgb_model, x_params,cv = 5,scoring=cs_scorer)
clf.fit(train_feats,y_train)
#saved = save_model(clf,'models/acd','model_'+cat)
print(clf.best_score_)
print(clf.best_params_)

In [150]:
pr = clf.predict(test_feats)

In [151]:
y_test['predicted sentiment'] = pr

In [154]:
cosine_similarity(y_test['sentiment'],y_test['predicted sentiment'])

array([[ 0.51963458]])

In [85]:
submission = pd.DataFrame({'id':X_test['id'],'sentiment score' : y_test['sentiment']})

In [86]:
submission.head()

,id,sentiment score
0,1144,0.691899
1,1145,0.191305
2,1146,-0.373309
3,1147,0.083100
4,1148,-0.658537


In [84]:
submission.to_json('submission.json',orient='records')